In [1]:
%load_ext autoreload
%autoreload 2

from molpy.builder.reacter_lammps import ReacterBuilder, ReactionTemplate, ReactantWrapper
from pathlib import Path
import molpy as mp
import molvis as mv
import numpy as np

workdir = Path("data/case2")

"""
Example usage: Generate reaction templates for nylon-6,6 synthesis.

This example demonstrates the condensation reaction between
adipic acid and hexamethylenediamine to form nylon-6,6.

Based on LAMMPS examples: https://github.com/lammps/lammps/tree/develop/examples/PACKAGES/reaction/nylon%2C6-6_melt
"""
# Load adipic acid structure (HOOC-(CH2)4-COOH)
adipic_acid_frame = mp.io.read_pdb(workdir/"adipicacid.pdb")
adipic_acid_struct = mp.Atomistic.from_frame(adipic_acid_frame, name="adipic_acid")
# adipic_acid_struct.gen_topo_items(is_angle=True, is_dihedral=True)

# Load hexamethylenediamine structure (H2N-(CH2)6-NH2)  
hexamethylenediamine_frame = mp.io.read_pdb(workdir/"hexamethylenediamine.pdb")
hexamethylenediamine_struct = mp.Atomistic.from_frame(hexamethylenediamine_frame, name="hexamethylenediamine")
# hexamethylenediamine_struct.gen_topo_items(is_angle=True, is_dihedral=True)

adipic_acid
hexamethylenediamine


In [3]:
builder = mp.builder.AmberToolsBuilder(workdir=workdir, conda_env="AmberTools25")
typifier = mp.builder.AmberToolsTypifier(workdir=workdir, conda_env="AmberTools25")

adipic_acid_struct = typifier.typify(adipic_acid_struct, is_prepi=True, is_frcmod=True, is_tleap=True)
hexamethylenediamine_struct = typifier.typify(hexamethylenediamine_struct, is_prepi=True, is_frcmod=True, is_tleap=True)
system = mp.System()
system.add_struct(adipic_acid_struct)
system.add_struct(hexamethylenediamine_struct)
system = typifier.parameterize("nylon66", system)
mp.io.write_lammps_forcefield(workdir/"nylon66.ff", system.forcefield)

Cleaned up temporary file: data/case2/adipic_acid/run_local.sh
Cleaned up temporary file: data/case2/adipic_acid/run_local.sh
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/prep to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/lib to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/parm to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.gaff
----- Source of /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.gaff done
Log file: ./leap.log
Loading parameters: /opt/conda/envs/AmberTools25/dat/leap/parm/gaff.dat
Reading title:
AMBER General Force Field for organic molecules (Version 1.81, May 2017)
Loading Prep file: /workspaces/molpy/examples/data/case2/adipic_acid/adipic_acid.prepi
Loading parameters: /workspaces/molpy/examples/data/case2/adipic_acid/adipic_acid.frcmod
Reading f

In [10]:
canvas = mv.Molvis()
canvas.draw_struct(adipic_acid_struct)
canvas.draw_struct(hexamethylenediamine_struct)

Molvis()

In [ ]:
adipic_acid_reactant = ReactantWrapper(adipic_acid_struct)
hexamethylenediamine_reactant = ReactantWrapper(hexamethylenediamine_struct)
aa_template = adipic_acid_reactant.extract_fragment(ReactionTemplate(
        init_atom=2,     # Carboxyl carbon
        edge_atom=7,      # OH atoms to be deleted
    ))

hd_template = hexamethylenediamine_reactant.extract_fragment(ReactionTemplate(
        init_atom=22,     # Amino nitrogen
        edge_atom=12,    # H atom to be deleted
))
mp.SpatialWrapper(hd_template).move([-10, 1, -2])

stp1_pre = ReactantWrapper(
    mp.Atomistic.concat("stp1_pre", [aa_template._wrapped, hd_template._wrapped])
)


get xyz
set xyz
[8.5064931, 0.8497441, -0.0599789] [ 8.5064931  1.8497441 -2.0599789]
[9.6242785, -0.1671602, -0.3119418] [ 9.6242785  0.8328398 -2.3119418]
[9.3483604, -0.7998677, -1.164561] [ 9.3483604  0.2001323 -3.164561 ]
[9.7152582, -0.8342415, 0.5541611] [ 9.7152582  0.1657585 -1.4458389]
[10.9800748, 0.466661, -0.6248764] [10.9800748  1.466661  -2.6248764]
[10.8971218, 1.1346342, -1.4898311] [10.8971218  2.1346342 -3.4898311]
[11.6830133, -0.3283371, -0.8988684] [11.6830133  0.6716629 -2.8988684]
[11.5272655, 1.2008392, 0.5084433] [11.5272655  2.2008392 -1.4915567]
[11.0262516, 2.0779635, 0.6264568] [11.0262516  3.0779635 -1.3735432]
[12.4997384, 1.4439058, 0.2943405] [12.4997384  2.4439058 -1.7056595]


In [32]:
stp1_post = stp1_pre(name="stp1_post")
stp1_post.def_bond(2, 15)

<Bond: O4-N2>

In [33]:
builder = ReacterBuilder(workdir=workdir, typifier=typifier)
stp2_pre = builder.build("stp1", stp1_pre, stp1_post, inits=[6, 8], edges=[2, 9])
canvas.draw_frame(stp2_pre.to_frame())

Frame(
  [atoms] q: shape=(18,)
  [atoms] id: shape=(18,)
  [atoms] mass: shape=(18,)
  [atoms] type: shape=(18,)
  [atoms] element: shape=(18,)
  [atoms] name: shape=(18,)
  [atoms] xyz: shape=(18, 3)
  [bonds] i: shape=(16,)
  [bonds] type: shape=(16,)
  [bonds] id: shape=(16,)
  [bonds] j: shape=(16,)
  [angles] id: shape=(25,)
  [angles] i: shape=(25,)
  [angles] type: shape=(25,)
  [angles] j: shape=(25,)
  [angles] k: shape=(25,)
  [dihedrals] id: shape=(29,)
  [dihedrals] i: shape=(29,)
  [dihedrals] type: shape=(29,)
  [dihedrals] l: shape=(29,)
  [dihedrals] j: shape=(29,)
  [dihedrals] k: shape=(29,)
)
Frame(
  [atoms] q: shape=(18,)
  [atoms] id: shape=(18,)
  [atoms] mass: shape=(18,)
  [atoms] type: shape=(18,)
  [atoms] element: shape=(18,)
  [atoms] name: shape=(18,)
  [atoms] xyz: shape=(18, 3)
  [bonds] i: shape=(17,)
  [bonds] type: shape=(17,)
  [bonds] id: shape=(17,)
  [bonds] j: shape=(17,)
  [angles] id: shape=(25,)
  [angles] i: shape=(25,)
  [angles] type: shap

Molvis()

In [ ]:
"bonds" in stp1_pre._wrapped.data

In [ ]:
stp2_pre["name"] = "stp2_pre"
stp2_post = stp2_pre(name="stp2_post").def_bond(6, 16)
stp2_post = stp2_post.del_bond(8, 16).del_bond(0, 6)

In [ ]:
reacted = builder.build("stp2", stp2_pre, stp2_post, inits=[6, 8], edges=[2, 9])

In [ ]:
import molpy.pack as mpk
box_size = np.array([40.0, 40.0, 40.0])  # 40 x 40 x 40 Angstrom box
box_origin = np.array([0.0, 0.0, 0.0])
packing_session = mpk.Session(packer="packmol")
box_constraint = mpk.InsideBoxConstraint(box_size, box_origin)
packing_session.add_target(
    frame=adipic_acid_struct.to_frame(),
    number=10,
    constraint=box_constraint
)
packing_session.add_target(
    frame=hexamethylenediamine_struct.to_frame(),
    number=10,
    constraint=box_constraint
)
frame = packing_session.optimize()
frame.box = mp.Box(box_size)
mp.io.write_lammps_data(workdir/"small_nylon_melt.data", frame)

In [ ]:
frame["bonds"]